In [5]:
import harp
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.io.img_tiles as cimgt
from cmcrameri import cm
import json, requests, os

In [6]:
start = str("2023-03-01") #first day of the analysed period
end = str("2023-03-31")   #last day of the analysed period

#querry generated by Data Explorer
url = f"https://datahub.creodias.eu/odata/v1/Products?$filter=((ContentDate/Start ge {start}T00:00:00.000Z and ContentDate/Start le {end}T23:59:59.999Z) and (Online eq true) and (OData.CSC.Intersects(Footprint=geography'SRID=4326;POLYGON ((13.44866 55.702105, 24.616418 55.287061, 24.505406 48.582106, 13.359851 49.556626, 13.44866 55.702105))')) and (((((((Attributes/OData.CSC.StringAttribute/any(i0:i0/Name eq 'productType' and i0/Value eq 'L2__NO2___')))) and (((Attributes/OData.CSC.StringAttribute/any(i0:i0/Name eq 'processingMode' and i0/Value eq 'OFFL')))) and (Collection/Name eq 'SENTINEL-5P'))))))&$expand=Attributes&$top=1000"

In [11]:
#get list of products (JSON)
products = json.loads(requests.get(url).text)

path_list = []                           #create an empty list to store the file names
for item in products['value']:
    path_list.append(item['S3Path'])     #append each S3Path (path) to the list
    print(item['S3Path'])                #print each S3Path (path) to the list

name_list = []                           #create an empty list to store the file names
for item in products['value']:
    name_list.append(item['Name'])       #append each Name (name) to the list
    print(item['Name'])                  #print each Name (name) to the list

/eodata/Sentinel-5P/TROPOMI/L2__NO2___/2023/03/30/S5P_OFFL_L2__NO2____20230330T093006_20230330T111136_28290_03_020500_20230401T015114.nc
/eodata/Sentinel-5P/TROPOMI/L2__NO2___/2023/03/31/S5P_OFFL_L2__NO2____20230331T105243_20230331T123414_28305_03_020500_20230402T031349.nc
/eodata/Sentinel-5P/TROPOMI/L2__NO2___/2023/03/31/S5P_OFFL_L2__NO2____20230331T091113_20230331T105243_28304_03_020500_20230402T011756.nc
/eodata/Sentinel-5P/TROPOMI/L2__NO2___/2023/03/30/S5P_OFFL_L2__NO2____20230330T111136_20230330T125307_28291_03_020500_20230401T033526.nc
/eodata/Sentinel-5P/TROPOMI/L2__NO2___/2023/03/29/S5P_OFFL_L2__NO2____20230329T094858_20230329T113029_28276_03_020500_20230331T021341.nc
/eodata/Sentinel-5P/TROPOMI/L2__NO2___/2023/03/28/S5P_OFFL_L2__NO2____20230328T114921_20230328T133052_28263_03_020500_20230330T042453.nc
/eodata/Sentinel-5P/TROPOMI/L2__NO2___/2023/03/27/S5P_OFFL_L2__NO2____20230327T120814_20230327T134944_28249_03_020500_20230329T043028.nc
/eodata/Sentinel-5P/TROPOMI/L2__NO2___/20

In [12]:
separator = '/'                                    #set separator between path and name
result_list = [name + separator + path for name,   #create a list of merged paths and names for each image
               path in zip(path_list, name_list)]
print(result_list)

['/eodata/Sentinel-5P/TROPOMI/L2__NO2___/2023/03/30/S5P_OFFL_L2__NO2____20230330T093006_20230330T111136_28290_03_020500_20230401T015114.nc/S5P_OFFL_L2__NO2____20230330T093006_20230330T111136_28290_03_020500_20230401T015114.nc', '/eodata/Sentinel-5P/TROPOMI/L2__NO2___/2023/03/31/S5P_OFFL_L2__NO2____20230331T105243_20230331T123414_28305_03_020500_20230402T031349.nc/S5P_OFFL_L2__NO2____20230331T105243_20230331T123414_28305_03_020500_20230402T031349.nc', '/eodata/Sentinel-5P/TROPOMI/L2__NO2___/2023/03/31/S5P_OFFL_L2__NO2____20230331T091113_20230331T105243_28304_03_020500_20230402T011756.nc/S5P_OFFL_L2__NO2____20230331T091113_20230331T105243_28304_03_020500_20230402T011756.nc', '/eodata/Sentinel-5P/TROPOMI/L2__NO2___/2023/03/30/S5P_OFFL_L2__NO2____20230330T111136_20230330T125307_28291_03_020500_20230401T033526.nc/S5P_OFFL_L2__NO2____20230330T111136_20230330T125307_28291_03_020500_20230401T033526.nc', '/eodata/Sentinel-5P/TROPOMI/L2__NO2___/2023/03/29/S5P_OFFL_L2__NO2____20230329T094858_2023

# from sentinel 2 notebook

In [17]:
# Timerange of data that we want to recieve
start_date = '2023-05-01'
end_date = '2023-07-31'
# Geometry in form of a BBOX
bbox = [10.678711, 50.035974,12.788086, 51.234407]

In [18]:
API_URL = "https://datahub.code-de.org/resto/api/collections/Sentinel5P/search.json?\
box={minx},{miny},{maxx},{maxy}\
&startDate={start_date}\
&completionDate={end_date}\
&sortParam=startDate\
&sortOrder=ascending\
&platform=S5P\
"

In [22]:
specified_url = API_URL.format(minx=bbox[0], miny=bbox[1], maxx=bbox[2], maxy=bbox[3], start_date=start_date, end_date=end_date)
print(specified_url)

https://datahub.code-de.org/resto/api/collections/Sentinel5P/search.json?box=10.678711,50.035974,12.788086,51.234407&startDate=2023-05-01&completionDate=2023-07-31&sortParam=startDate&sortOrder=ascending&platform=S5P


In [24]:
#get list of products (JSON)
products2 = json.loads(requests.get(specified_url).text)

path_list2 = []                           #create an empty list to store the file names
for item in products2['value']:
    path_list.append(item['S3Path'])     #append each S3Path (path) to the list
    print(item['S3Path'])                #print each S3Path (path) to the list

name_list = []                           #create an empty list to store the file names
for item in products2['value']:
    name_list.append(item['Name'])       #append each Name (name) to the list
    print(item['Name'])                  #print each Name (name) to the list

KeyError: 'value'

# back 2 business

In [10]:
N = 55.00
S = 49.00
W = 14.00
E = 25.00

SR = 0.01

A = (N-S)/SR + 1
B = (E-W)/SR + 1

operations = ";".join([
    "tropospheric_NO2_column_number_density_validity>75",             #keep pixels wich qa_value > 0.75
    "derive(surface_wind_speed {time} [m/s])",                        #get surafe wind speed expressed in [m/s]
    "surface_wind_speed<5",                                           #keep pixels wich wind_spped < 5 [m/s]
    #keep variables defined below
    "keep(latitude_bounds,longitude_bounds,datetime_start,datetime_length,tropospheric_NO2_column_number_density, surface_wind_speed)",
    "derive(datetime_start {time} [days since 2000-01-01])",          #get start time of the acquisition
    "derive(datetime_stop {time}[days since 2000-01-01])",            #get end time of the acquisition
    "exclude(datetime_length)",                                       #exclude datetime lenght
    f"bin_spatial({int(A)},{int(S)},{float(SR)},{int(B)},{int(W)},{float(SR)})",                       #define bin spatial (details below)
    "derive(tropospheric_NO2_column_number_density [Pmolec/cm2])",    #convert the NO2 units to 10^15 molec/cm^2
    "derive(latitude {latitude})",                                    #get latitude
    "derive(longitude {longitude})",                                  #get longitude
    "count>0"
])

In [ ]:
reduce_operations=";".join([
    "squash(time, (latitude, longitude, latitude_bounds, longitude_bounds))",
    "bin()"
])

In [25]:
mean_no2 = harp.import_product(result_list, operations, reduce_operations=reduce_operations)

NameError: name 'reduce_operations' is not defined

In [ ]:
harp.export_product(mean_no2, 'mean_no2_2023_03.nc') #name of variable, name of new created file with extension

In [ ]:
#select NO2 pollution
no2 = mean_no2.tropospheric_NO2_column_number_density.data
#select desription (name) of variable
no2_description = mean_no2.tropospheric_NO2_column_number_density.description
#select units of variable
no2_units = mean_no2.tropospheric_NO2_column_number_density.unit

#define cooridantes grid
gridlat = np.append(mean_no2.latitude_bounds.data[:,0], mean_no2.latitude_bounds.data[-1,1])
gridlon = np.append(mean_no2.longitude_bounds.data[:,0], mean_no2.longitude_bounds.data[-1,1])

#define legend
colortable = cm.roma_r  #colors
vmin = 0                #min value
vmax = 10               #max value

In [ ]:
boundaries=[14, 25, 49, 55.0]                       #define boundaries (W, E, S, N)

fig = plt.figure(figsize=(10,10))                   #size of the figure
bmap=cimgt.Stamen(style='toner-lite')               #set background
ax = plt.axes(projection=bmap.crs)
ax.set_extent(boundaries,crs = ccrs.PlateCarree())

zoom = 10
ax.add_image(bmap, zoom)

img = plt.pcolormesh(gridlon, gridlat,
                     no2[0,:,:], vmin=vmin, vmax=vmax,
                     cmap=colortable,
                     transform=ccrs.PlateCarree(),alpha = 0.55)
ax.coastlines()

cbar = fig.colorbar(img, ax=ax,orientation='horizontal', fraction=0.04, pad=0.1)
cbar.set_label(f'{no2_description} [{no2_units}]')
cbar.ax.tick_params(labelsize=14)

plt.show()
